In [1]:
import findspark
findspark.init()
import os
import pyspark
import sys
from pyspark.sql import SparkSession

In [2]:
## Start Spark Session
spark = SparkSession.builder.appName('app_name') \
        .config('spark.jars.packages','org.apache.hudi:hudi-spark3.3-bundle_2.12:0.13.1') \
        .config('spark.serializer','org.apache.spark.serializer.KryoSerializer') \
        .config('spark.sql.extensions','org.apache.spark.sql.hudi.HoodieSparkSessionExtension') \
        .config('spark.sql.catalog.spark_catalog','org.apache.spark.sql.hudi.catalog.HoodieCatalog') \
        .getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/opt/anaconda3/envs/spark_hudi/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
23/09/01 13:40:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Ivy Default Cache set to: /Users/anshumanr/.ivy2/cache
The jars for the packages stored in: /Users/anshumanr/.ivy2/jars
org.apache.hudi#hudi-spark3.3-bundle_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-73fdafb8-a20e-4445-b198-a5572a3edb9c;1.0
	confs: [default]
	found org.apache.hudi#hudi-spark3.3-bundle_2.12;0.13.1 in central
:: resolution report :: resolve 66ms :: artifacts dl 1ms
	:: modules in use:
	org.apache.hudi#hudi-spark3.3-bundle_2.12;0.13.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submi

Spark Running


## Hudi Upsert

In [3]:
tablename='customers_upsert'                # 1 Million Records
basePath = "/Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/customers_upsert" 

In [4]:
df = spark.read.format('parquet').options(inferSchema = 'True',header='True').load("/Users/anshumanr/Downloads/data/customers.parquet")

In [5]:
hudi_options = {
    'hoodie.table.name': tablename,
    'hoodie.datasource.write.recordkey.field': 'customer_id',
    'hoodie.datasource.write.partitionpath.field': 'gender',
    'hoodie.datasource.write.table.name': tablename,
    'hoodie.datasource.write.operation': 'upsert',
    'hoodie.datasource.write.precombine.field':'age'
}

In [6]:
%%timeit -n 1 -r 1
df.write.format("hudi"). \
options(**hudi_options).mode("overwrite").save(basePath)

23/09/01 13:40:33 WARN DFSPropertiesConfiguration: Cannot find HUDI_CONF_DIR, please set it as the dir of hudi-defaults.conf
23/09/01 13:40:33 WARN DFSPropertiesConfiguration: Properties file file:/etc/hudi/conf/hudi-defaults.conf not found. Ignoring to load props file
23/09/01 13:40:34 WARN HoodieBackedTableMetadata: Metadata table was not found at path /Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/customers_upsert/.hoodie/metadata


23/09/01 13:40:38 WARN MemoryStore: Not enough space to cache rdd_22_0 in memory! (computed 7.7 MiB so far)
23/09/01 13:40:38 WARN MemoryStore: Not enough space to cache rdd_22_2 in memory! (computed 7.7 MiB so far)
23/09/01 13:40:38 WARN BlockManager: Persisting block rdd_22_0 to disk instead.
23/09/01 13:40:38 WARN BlockManager: Persisting block rdd_22_2 to disk instead.
23/09/01 13:40:38 WARN MemoryStore: Not enough space to cache rdd_22_1 in memory! (computed 7.7 MiB so far)
23/09/01 13:40:38 WARN BlockManager: Persisting block rdd_22_1 to disk instead.


23/09/01 13:40:40 WARN MemoryStore: Not enough space to cache rdd_39_3 in memory! (computed 5.1 MiB so far)
23/09/01 13:40:40 WARN BlockManager: Persisting block rdd_39_3 to disk instead.


23/09/01 13:40:43 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


# WARNING: Unable to get Instrumentation. Dynamic Attach failed. You may add this JAR as -javaagent manually, or supply -Djdk.attach.allowAttachSelf
# WARNING: Unable to attach Serviceability Agent. Unable to attach even with module exceptions: [org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed.]
23/09/01 13:40:56 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-hbase.properties,hadoop-metrics2.properties


23.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Hudi Insert

In [7]:
tablename='customers_insert'                # 1 Million Records
basePath = "/Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/customers_insert" 

In [8]:
df = spark.read.format('parquet').options(inferSchema = 'True',header='True').load("/Users/anshumanr/Downloads/data/customers.parquet")

In [9]:
hudi_options = {
    'hoodie.table.name': tablename,
    'hoodie.datasource.write.recordkey.field': 'customer_id',
    'hoodie.datasource.write.partitionpath.field': 'gender',
    'hoodie.datasource.write.table.name': tablename,
    'hoodie.datasource.write.operation': 'insert',
    'hoodie.datasource.write.precombine.field':'age'
}

In [10]:
%%timeit -n 1 -r 1
df.write.format("hudi"). \
options(**hudi_options).mode("overwrite").save(basePath)

23/09/01 13:40:57 WARN HoodieBackedTableMetadata: Metadata table was not found at path /Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/customers_insert/.hoodie/metadata


9.56 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Hudi BulkInsert

In [11]:
tablename='customers_bulkinsert'                # 1 Million Records
basePath = "/Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/customers_bulkinsert" 

In [12]:
df = spark.read.format('parquet').options(inferSchema = 'True',header='True').load("/Users/anshumanr/Downloads/data/customers.parquet")

In [13]:
hudi_options = {
    'hoodie.table.name': tablename,
    'hoodie.datasource.write.recordkey.field': 'customer_id',
    'hoodie.datasource.write.partitionpath.field': 'gender',
    'hoodie.datasource.write.table.name': tablename,
    'hoodie.datasource.write.operation': 'bulk_insert',
    'hoodie.datasource.write.precombine.field':'age'
}

In [14]:
%%timeit -n 1 -r 1
df.write.format("hudi"). \
options(**hudi_options).mode("overwrite").save(basePath)

23/09/01 13:41:06 WARN HoodieBackedTableMetadata: Metadata table was not found at path /Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/customers_bulkinsert/.hoodie/metadata
23/09/01 13:41:06 WARN HoodieBackedTableMetadata: Metadata table was not found at path /Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/customers_bulkinsert/.hoodie/metadata


23/09/01 13:41:09 WARN HoodieBackedTableMetadata: Metadata table was not found at path /Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/customers_bulkinsert/.hoodie/metadata
23/09/01 13:41:09 WARN HoodieBackedTableMetadata: Metadata table was not found at path /Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/customers_bulkinsert/.hoodie/metadata
3.96 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
